In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
directory = "./Logs/"
save_path = "./Saved/"

from load_logs import *
from evaluation import *
from features import *
from visualize import *

logs, log_labels = read_logs_and_labels("./Saved/logs.txt", "./Saved/labels.txt")

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
import os
import pickle
import sys
import random
import hdbscan

from scipy import sparse
from math import sqrt
from sklearn import metrics
from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn.datasets import make_blobs
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans

In [6]:
#Time to define necessary functions
#Since this is based on a knn idea we will need to calculate euclidean distance

In [64]:
#Euclidan distance function
def eucDist(x,y):
    dist = 0.0
    for i in range (len(y)-1):
        dist += (x[i] - y[i])**2
    return sqrt(dist)

In [65]:
#Nearest neighbour - gets x number of nearest nodes
def nearest(x, y_row, neighbors, labels): #specify number of neighbours to return
    distances = list()
    for i in range(len(x)): #loop through x, calculate distance and sort by distance, will allow us to get nearest neighbor
        x_row = x[i]
        dist = eucDist(y_row, x_row)
        nodeLabels = list()
        for lab in labels:
            nodeLabels.append(lab[i])
        distances.append((x_row, dist, nodeLabels))
        distances.sort(key=lambda tup: tup[1])
    neighborsList = list()
    neighborsLabels = list()
    for i in range(neighbors): #27093
        neighborsList.append(distances[i][0])
        neighborsLabels.append(distances[i][2])
    return neighborsList, neighborsLabels

In [81]:
#Current and nearest are same cluster then true otherwise false
def checkClus(x,y):
    boolArr = list()
    for i in range(len(x)):
        if (x[i] == y[i]):
            boolArr.append(True)
        else:
            boolArr.append(False)
    return boolArr

In [83]:
def consensus(boolArr): #form boolean array from checkClus, if majority are true then the node should be in the cluster, otherwise not
    trueCount = 0
    total = len(boolArr)
    for i in boolArr:
        if i == True:
            trueCount = trueCount + 1
    if (trueCount*100)/(total*100) > 50:
        #pass vote of node being in cluster
        return True
    else:
        #pass the false vote - node is not in cluster
        return False

In [80]:
def takeConsensus(X, labels):
    conList = list()
    X_ = X
    for i in range(len(X)):
        boolArr = list()
        near, tempLab = nearest(X_, X[i], 3, labels)
        X_ = np.delete(X, i, axis = 0) #This will exclude the current node from our next nearest calculations
        boolArr1 = checkClus(tempLab[0],tempLab[1])
        boolArr2 = checkClus(tempLab[0],tempLab[2])
        if consensus(boolArr1):
            conList.append(tempLab[0][0]) #need to sort this out such that we assign a valid cluster number
        elif consensus(boolArr2): #here we check its second nearest neighbor before reclustering
            conList.append(tempLab[1][0])
        else: #recluster somehow
            conList.append(labels[i][0])
    return conList

In [ ]:
#def recluster(): #For now we will repredict the node with Kmeans

In [11]:
labels_ = log_labels
            
X = get_features(logs, 2, 70)
X = X.toarray()

idxs = np.where(np.all(X == 0, axis=1))

X = np.delete(X, idxs, axis=0)
labels_ = np.delete(labels_, idxs)
## K Means Labels
kmeans = KMeans(init="k-means++", n_clusters=10,random_state=0).fit(X)
kLabels = kmeans.labels_

## DBSCAN Labels
DBmodel = DBSCAN(eps=0.4,min_samples=200,algorithm="auto",metric="cosine")
DBmodel.fit(X)
dbLabels = DBmodel.labels_

## HDBScan Label
Hmodel = hdbscan.HDBSCAN(min_cluster_size=500,min_samples=100,cluster_selection_epsilon=0.5,
                                        metric="euclidean",algorithm='best')
Hmodel.fit(X)
hdcLabels = Hmodel.labels_

In [82]:
#testing
labels = list()
labels.append(kLabels)
labels.append(dbLabels)
labels.append(hdcLabels)

cLabels = takeConsensus(X,labels)



0.0
1.3908456115787189
1.3908926728968498


NameError: name 'true' is not defined

In [ ]:
results = evaluate_clustering('Consensus Bigram Clustering', X, labels_, cLabels)
print(results)